In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Read Bronze table

In [0]:
df = spark.table("workspace.bronze.erp_loc_a101")

#Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))
     

In [0]:
df.display()

#Customer ID Cleanup

In [0]:
df = df.withColumn("cid", regexp_replace(col("cid"), "-", ""))

#Country Normalization

In [0]:
df = df.withColumn("cntry",
                   when (col("cntry")== "DE", "Germany")
                   .when (col("cntry").isin("US","USA"),"United Satates")
                    .when ((col("cntry") == "") | col("cntry").isNull(), "n/a")
                    .otherwise(col("cntry"))
                    
                    
)

In [0]:
df.display()

In [0]:
RENAME_MAP = {
    "cid": "customer_number",
    "cntry": "country"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:

df.limit(10).display()

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")

In [0]:


%sql
SELECT * FROM workspace.silver.erp_customer_location LIMIT 10